# QUERY

In [1]:
import pymongo
import pprint
from pymongo import MongoClient
from bson.objectid import ObjectId

client = MongoClient('localhost', 27017)
db = client.FormulaTweet
piloti = db.piloti
scuderie = db.scuderie
lap = db.lap
tweet = db.tweet

- Pilota con più tweet
- Pilota che ha twittato di più
- Scuderia con più tweet
- Scuderia che ha twittato di più
- Tweet del canale ufficiale più retwittato
- Evento relativo al Tweet del canale ufficiale più retwittato
- Quanti tweet sono stati effettuati durante la gara


**Pilota con più Tweet**

In [2]:
q1 = piloti.aggregate([
    {'$project':
         {
           '_id': 0,
           'Driver': "$name",
           'Numero tweet utenti': { '$size': '$Tweet utenti' }
         }
     },
    {'$sort': {'Numero tweet utenti': -1}},
    {'$limit' : 1}
]
)
for doc in q1:
    pprint.pprint(doc)

{'Driver': 'Sebastian Vettel', 'Numero tweet utenti': 10416}


**Pilota che ha Twittato di più**

In [3]:
q2 = piloti.aggregate([
    {'$project':
         {
           '_id': 0,
           'Driver': "$name",
           'Numero tweet pilota': {'$cond': [{'$not': '$Tweet pilota'}, 0, {'$size': '$Tweet pilota' }]}
         }
     },
    {'$sort': {'Numero tweet pilota': -1}},
    {'$limit' : 1}
]
)
for doc in q2:
    pprint.pprint(doc)

{'Driver': 'Romain Grosjean', 'Numero tweet pilota': 12}


**Scuderia con più Tweet**

In [4]:
q3 = scuderie.aggregate([
    {'$project':
         {
           '_id': 0,
           'Scueria': "$constructorName",
           'Numero tweet utenti': { '$size': '$Tweet utenti' }
         }
     },
    {'$sort': {'Numero tweet utenti': -1}},
    {'$limit' : 1}
]
)
for doc in q3:
    pprint.pprint(doc)

{'Numero tweet utenti': 3003, 'Scueria': 'Ferrari'}


**Scuderia che ha twittato di più**

In [5]:
q4 = scuderie.aggregate([
    {'$project':
         {
           '_id': 0,
           'Scuderia': "$constructorName",
           'Numero tweet scuderia': {'$cond': [{'$not': '$Tweet scuderia'}, 0, {'$size': '$Tweet scuderia' }]}
         }
     },
    {'$sort': {'Numero tweet scuderia': -1}},
    {'$limit' : 1}
]
)
for doc in q4:
    pprint.pprint(doc)

{'Numero tweet scuderia': 70, 'Scuderia': 'Racing Point'}


**Qual'è il tweet del canale ufficiale con più retweet**

In [6]:
#funziona
q5 = lap.aggregate([
    {
        '$match':{'Tweet official channel': {'$ne': None}}
        
    },
    {
        '$lookup':{
                'from': 'tweet',
                'localField': 'Tweet official channel.id',
                'foreignField': 'retweet.id',
                'as': 'id_official'
            }},
    {
        '$unwind': '$id_official'
    },
    {
        '$group':{
            '_id': '$Tweet official channel.id',
            'text': {'$first': '$Tweet official channel.text'},
            'lap': {'$first': '$Lap'},
            'total_retweet': {'$sum': 1}}
    },
    {
        '$sort':{'total_retweet': -1}
    }
])

In [7]:
for doc in q5:
    pprint.pprint(doc)

{'_id': '1155494053603172358',
 'lap': 63,
 'text': 'LAP 63/64\n'
         '\n'
         'Vettel passes Stroll to take P3\n'
         '\n'
         'Seconds later Vettel passes Kvyat to take P2\n'
         '\n'
         'Reminder: he started from P20 😮\n'
         '\n'
         '#F1 #GermanGP 🇩🇪 https://t.co/gez3Mvqml3',
 'total_retweet': 698}
{'_id': '1155492416432029696',
 'lap': 57,
 'text': 'LAP 57/64: SAFETY CAR\n'
         '\n'
         'The race goes from bad to worse for Mercedes 👀\n'
         '\n'
         "Valtteri Bottas' race is over as pushes beyond the limit in the race "
         'for the final podium place 💥\n'
         '\n'
         'Lewis Hamilton is down in P14\n'
         '\n'
         '#F1 #GermanGP 🇩🇪 https://t.co/NYu9T6MNrA',
 'total_retweet': 688}
{'_id': '1155485569784459266',
 'lap': 40,
 'text': 'LAP 40/64\n'
         '\n'
         'Heartbreak for @HulkHulkenberg as he crashes into the barriers and '
         "can't recover 💔\n"
         '\n'
         'With a

**Qual è l'evento con più retweet**

In [8]:
q6 = lap.aggregate([
    {
        '$match':{'Tweet official channel': {'$ne': None}, 'Evento rilevato':{'$exists': True}}
    },
    {
        '$lookup':{
                'from': 'tweet',
                'localField': 'Tweet official channel.id',
                'foreignField': 'retweet.id',
                'as': 'id_official'
            }},
    {
        '$unwind': '$id_official'
    },
    {
        '$group':{
            '_id': '$Tweet official channel.id',
            'text': {'$first': '$Tweet official channel.text'},
            'lap': {'$first': '$Lap'},
            'total_retweet': {'$sum': 1},
            'evento': {'$first': '$Evento rilevato'}}
    },
    {
        '$sort':{'total_retweet': -1}
    }
])

In [9]:
for doc in q6:
    pprint.pprint(doc)

{'_id': '1155492416432029696',
 'evento': {'Driver_Id': 822,
            'Evento': 'Accident',
            'Soggetto': 'Valtteri Bottas'},
 'lap': 57,
 'text': 'LAP 57/64: SAFETY CAR\n'
         '\n'
         'The race goes from bad to worse for Mercedes 👀\n'
         '\n'
         "Valtteri Bottas' race is over as pushes beyond the limit in the race "
         'for the final podium place 💥\n'
         '\n'
         'Lewis Hamilton is down in P14\n'
         '\n'
         '#F1 #GermanGP 🇩🇪 https://t.co/NYu9T6MNrA',
 'total_retweet': 688}
{'_id': '1155485569784459266',
 'evento': {'Driver_Id': 807,
            'Evento': 'Accident',
            'Soggetto': 'Nico Hülkenberg'},
 'lap': 40,
 'text': 'LAP 40/64\n'
         '\n'
         'Heartbreak for @HulkHulkenberg as he crashes into the barriers and '
         "can't recover 💔\n"
         '\n'
         'With a first F1 podium in sight, his race is over\n'
         '\n'
         '#F1 #GermanGP 🇩🇪 https://t.co/cS8IybB7sT',
 'total_retweet'

**Quanti tweet sono stati effettuati durante la gara**

In [10]:
import datetime
q7 = tweet.count_documents({
    'created_at': {
        '$gte': datetime.datetime(2019, 7, 28, 13, 20, 1, tzinfo=datetime.timezone.utc),
        '$lte': datetime.datetime(2019, 7, 28, 15, 5, 0, tzinfo=datetime.timezone.utc)
    }
})

q7

140333